In [ ]:
from pathlib import Path
import sys

BASE_DIR = Path('.').resolve().parent
sys.path.append(str(BASE_DIR))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d

In [ ]:
%load_ext autoreload
%autoreload 2

from scripts_old.plotter import Plotter
import scripts_old.postprocessing as pp

In [ ]:
CURRENT_DATA_SLOW = '2022-09-25_velocity_scaling_HV_2e-5_T_01e-4'
PLOT_FILENAME_POSTFIX_SLOW = 'cooling_slow'

In [ ]:
PATH_TO_CURRENT_DATA_SLOW = BASE_DIR / 'data' / CURRENT_DATA_SLOW
PATH_TO_CURRENT_PLOTS = BASE_DIR / 'plots' / 'article'
PATH_TO_CURRENT_DATA_SLOW

In [ ]:
setups_slow = []
rdf_slow = pp.RadialDistributionFunctionPP(path_to_data=str(PATH_TO_CURRENT_DATA_SLOW))

for filename in PATH_TO_CURRENT_DATA_SLOW.iterdir():
    if filename.match('rdf*.csv'):
        parts = filename.stem.split('_')
        setup = {
            'temperature': float(parts[2]),
            'pressure': float(parts[4]) if 'P' in parts else None,
            'heating_velocity': float(parts[6]) if 'HV' in parts else None,
        }
        setups_slow.append(setup)

setups_slow = setups_slow[::-1]
for setup_number, setup in enumerate(setups_slow):
    print(setup)
    filename_postfix = pp.get_filename_postfix(setup)
    rdf_slow.append(filename_postfix=filename_postfix)
    rdf_slow.rename_column(column_name='rdf', setup_number=setup_number)

rdf_slow.data

In [ ]:
rdf_table = rdf_slow.get_rdf_table()
rdf_table

In [ ]:
rdf_0 = rdf_slow.data[['radius', 'setup_0']].rename(columns={'setup_0': 'rdf'})
non_zero_rdf_0 = rdf_0.loc[rdf_0.rdf > 0]
non_zero_rdf_0

In [ ]:
plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=0,
        right=5,
        bottom=-0.1,
        top=2.5,
    ),
    labels=('radius', 'rdf'),
)

step = 15

plotter.ax.plot(
    rdf_0.radius.values, 
    rdf_0.rdf.values,
    label=r'$T=156$ K',
    linewidth=1,
)
plotter.ax.scatter(
    non_zero_rdf_0.radius.values[::step], 
    non_zero_rdf_0.rdf.values[::step],
#     label=r'$g^{(j)}(T)$',
    s=20,
)
fontsize = 10
plotter.ax.text(s=r'$g^{(1)}(T)$', x=1, y=0.05, fontsize=fontsize)
plotter.ax.text(s=r'$g^{(2)}(T)$', x=0.2, y=1.4, fontsize=fontsize)
plotter.ax.text(s=r'$g^{(3)}(T)$', x=1.25, y=2, fontsize=fontsize)
plotter.ax.text(s=r'$g^{(4)}(T)$', x=1.3, y=1.25, fontsize=fontsize)
plotter.ax.text(s=r'$g^{(j)}(T)$', x=3.05, y=1.1, fontsize=fontsize)
plotter.ax.text(s=r'$g^{(n_r)}(T)$', x=4.2, y=0.7, fontsize=fontsize)
plotter.ax.arrow(x=4.6, y=0.83, dx=0.23, dy=0.11, width=0.01)
plotter.set_minor_locators(0.2, 0.1)
plotter.get_legend()
plotter.save_plot(filename=f'rdf_parameters_example.png')
plt.show()

In [ ]:
scales = np.arange(1, 100, 4)
steps = 0.01 * scales
sums = [0.01 * scale * non_zero_rdf_0.rdf[::scale].sum() for scale in scales]
integral = non_zero_rdf_0.rdf.sum() * 0.01


plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=0,
        right=1,
        bottom=-0.01,
        top=0.25,
    ),
    labels=(r'$\Delta r$ (units of $\sigma$)', r'$|\hat{S}(T)-S(T)|~/~S(T)$'),
)

deltas = abs(integral - sums) / integral
plotter.ax.plot(steps, deltas, 'o--', color='black', label=r'$T=156$ K', linewidth=1)
dot_index = 8
plotter.ax.plot(np.linspace(-0.1, steps[dot_index], 40), deltas[dot_index] * np.ones(40), '--', color='black', linewidth=1)
plotter.ax.plot(steps[dot_index] * np.ones(5), np.linspace(-0.1, deltas[dot_index], 5), '--', color='black', linewidth=1)

plotter.set_minor_locators(0.05, 0.01)
plotter.get_legend()

plotter.save_plot(filename=f'sums_step_dependence.png')
plt.show()
steps[dot_index], deltas[dot_index]

In [ ]:
temperatures = np.array([
    setups_slow[value]['temperature']
    for value
    in rdf_table.index.str[6:].values.astype(np.int32)
])[:-1]
temperatures

In [ ]:
samples_train_indices = np.arange(temperatures.shape[0])
samples_test_indices = np.arange(temperatures.shape[0], temperatures.shape[0])

train_samples = temperatures[samples_train_indices]
test_samples = temperatures[samples_test_indices]
train_samples, test_samples

In [ ]:
train_targets_1 = rdf_table[1.5].values[samples_train_indices]
train_targets_2 = rdf_table[2.0].values[samples_train_indices]
train_targets_3 = rdf_table[2.5].values[samples_train_indices]


plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=84,
        right=156,
        bottom=0.4,
        top=1.6,
    ),
    labels=(r'$T$ (K)', fr'$g(T;r_j)$'),
)
kelvins = train_samples * 119.8
plotter.ax.scatter(kelvins, train_targets_1, label=fr'$r_j=1.5\sigma$', s=20)
plotter.ax.scatter(kelvins, train_targets_2, label=fr'$r_j=2.0\sigma$', s=20)
plotter.ax.scatter(kelvins, train_targets_3, label=fr'$r_j=2.5\sigma$', s=20)
plotter.set_minor_locators(4, 0.04)
plotter.ax.text(s=r'$\gamma=10^8$ K/s', x=130, y=0.5, fontsize=10, bbox=dict(boxstyle='round', facecolor='lightyellow'))
plotter.get_legend()
plotter.save_plot('regression_examples')
plt.show()

In [ ]:
np.random.seed(0)
features = np.array([90, 92, 94, 96, 98, 101, 103, 106, 108])
labels = 0.85 + np.random.random(9) * 0.3

In [ ]:
plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=90,
        right=110,
        bottom=0.85,
        top=1.15,
    ),
    labels=(r'$T$ (K)', fr'$g^{{(j)}}(T)$'),
)
plotter.ax.scatter(features[:3], labels[:3], s=20, color='black')
plotter.ax.scatter(features[7:], labels[7:], s=20, color='black')
plotter.ax.scatter(features[3:7], labels[3:7], s=20, color='green')
plotter.ax.scatter(100, labels[3:7].mean(), s=20, color='red')
plotter.set_minor_locators(2, 0.02)
plotter.ax.scatter(100, labels[3:7].mean(), s=20, color='red')
plotter.ax.text(s=r'$k=4$', x=105, y=0.9)
plotter.save_plot('knn_example')
plt.show()

In [ ]:
PATH_TO_LJ_PHASES = BASE_DIR / 'data' / '2022-12-10_lj_phases'

solid = pd.read_csv(PATH_TO_LJ_PHASES / 'rdf_T_0.10015_P_-0.58748_HV_0.00002_.csv', sep=';')

liquid = pd.read_csv(PATH_TO_LJ_PHASES / 'rdf_T_0.66765_P_0.00568_HV_0.00002_.csv', sep=';')

gas = pd.read_csv(PATH_TO_LJ_PHASES / 'rdf_T_2.48628_P_0.08435_HV_0.00002_.csv', sep=';')

plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=0,
        right=5,
        bottom=-0.1,
        top=12,
    ),
    labels=('radius', 'rdf'),
)

step = 15

plotter.ax.plot(solid.radius.values, solid.rdf.values, label=r'solid', linewidth=1, color=(0, 1, 0))
plotter.ax.plot(liquid.radius.values, liquid.rdf.values, ':', label=r'liquid', linewidth=1, color=(1, 0, 0))
plotter.ax.plot(gas.radius.values, gas.rdf.values, '--', label=r'gas', linewidth=1, color=(0, 0, 1))
plotter.set_minor_locators(0.2, 0.4)
plotter.get_legend()
plotter.save_plot(filename=f'rdf_for_lj_phases.png')
plt.show()

In [ ]:
np.random.seed(0)
split_temperatures = np.linspace(0, 1, 9)
dt_values = np.random.uniform(size=8)

plotter = Plotter(
    path_to_plots=PATH_TO_CURRENT_PLOTS,
    limits=dict(
        left=0,
        right=1,
        bottom=0,
        top=1,
    ),
)
for i, split_temperature in enumerate(split_temperatures[:-1]):
    temperatures = np.linspace(split_temperature, split_temperatures[i + 1], 2)
    plotter.ax.plot(temperatures, np.ones(2) * dt_values[i], color='black')
    plotter.ax.plot(np.ones(2) * split_temperature, np.arange(2), '--', color='black', linewidth=1)
    plotter.ax.text(
        s=fr'$g_{i + 1}$', 
        x=(split_temperature+split_temperatures[i + 1]) / 2 - 0.03, 
        y=dt_values[i] + 0.02,
    )
plotter.ax.set_ylabel(fr'$g^{{(j)}}(T)$')
plotter.ax.set_xticks(split_temperatures[1:-1])
plotter.ax.set_xticklabels(
    [
    r'$T_1^{(2)}$',
    r'$T_1^{(1)}$',
    r'$T_2^{(2)}$',
    r'$T_1^{(0)}$',
    r'$T_3^{(2)}$',
    r'$T_2^{(1)}$',
    r'$T_4^{(2)}$',
    ],
)
plotter.ax.set_yticks([0, 1])
plotter.ax.set_yticklabels(['', ''])
plotter.ax.text(s='(b)', x=-.2, y=0.9, fontsize=14)
plotter.save_plot(filename=f'decision_tree_values.png')
plt.show()